# Install necessary packages before running the code

Load model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("tarudesu/ViHateT5-base-HSD")
model = AutoModelForSeq2SeqLM.from_pretrained("tarudesu/ViHateT5-base-HSD")

Inference function

In [ ]:
def generate_output(input_text, prefix):
    # Add prefix
    prefixed_input_text = prefix + ': ' + input_text

    # Tokenize input text
    input_ids = tokenizer.encode(prefixed_input_text, return_tensors="pt")

    # Generate output
    output_ids = model.generate(input_ids, max_length=256)

    # Decode the generated output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output_text

sample = 'Tôi ghét bạn vl luôn!'
prefix = 'hate-spans-detection'

result = generate_output(sample, prefix)
result

Find index of hate spans

In [ ]:
import unicodedata

def connect_values(input_list, input_text): # For [hate]abc[hate] [hate]xyz[hate] in the sample 'abc xyz' => Merge the index and insert the idx of the space between two consecutive words

    input_list = ast.literal_eval(input_list) # '[]' => []

    modified_list = []
    temp_list = []

    for i in range(len(input_list) - 1):
        modified_list.append(input_list[i])

        if input_list[i + 1] - input_list[i] == 2:
            difference = input_list[i + 1] - input_list[i]
            temp_list.extend(input_list[i] + j for j in range(1, difference))

    # Check each value in temp_list, if input_text[temp_list[i]] is not a space, remove temp_list[i]
    for idx in temp_list:
        if input_text[idx] != " ":
            temp_list.remove(idx)

    modified_list.extend(temp_list)
    modified_list.append(input_list[-1])

    return sorted(modified_list)


def find_and_extract_substrings(original_str, input_str):

    start_tag = '[hate]'
    end_tag = '[hate]'

    input_str = unicodedata.normalize('NFC', input_str.lower())
    original_str = unicodedata.normalize('NFC', original_str.lower())

    # Extract substrings
    substrings = []
    start_index = input_str.find(start_tag)
    while start_index != -1:
        end_index = input_str.find(end_tag, start_index + len(start_tag))
        if end_index != -1:
            substrings.append(input_str[start_index + len(start_tag):end_index])
            start_index = input_str.find(start_tag, end_index + len(end_tag))
        else:
            break

    if not substrings:
        return ''

    # Find indices in the original string and merge into one list
    indices_list = []
    for substring in substrings:
        start_index = original_str.find(substring)
        while start_index != -1:
            indices_list.extend(list(range(start_index, start_index + len(substring))))
            start_index = original_str.find(substring, start_index + 1)

    deduplicated_sorted_indices_list = sorted(set(indices_list))

    deduplicated_sorted_indices_list = connect_values(str(deduplicated_sorted_indices_list), original_str)

    return str(deduplicated_sorted_indices_list)

result_indices_str = find_and_extract_substrings(sample, result)
result_indices_str